In [1]:
import gymnasium as gym
import tensorflow as tf
from tensorflow import keras
import numpy as np

2023-04-21 22:55:33.851711: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from tensorflow.keras.initializers import Initializer
keras.Sequential()

2023-04-21 22:55:40.499002: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
#env = gym.make('CartPole-v1')
env = gym.make(
            id="CartPole-v1",  # Choose one of the existing environments
            render_mode="rgb_array",  # The set of supported modes varies per environment. (And some third-party environments may not support rendering at all.)
        )

Action Space is a [Discrete](https://gymnasium.farama.org/api/spaces/fundamental/#gymnasium.spaces.Discrete) type
A space consisting of finitely many elements.
In CartPole-v1, it has two actions: left or right

In [4]:
print(type(env.action_space))
print(f"Action space size: {env.action_space.n}")
print(f"Action space start value: {env.action_space.start}")
env.action_space

<class 'gymnasium.spaces.discrete.Discrete'>
Action space size: 2
Action space start value: 0


Discrete(2)

Observation space is a 4 dimension state: 
- Cart Position
- Cart Velocity
- Pole Angle
- Pole Velocity at tip

In [5]:
print(type(env.observation_space))
# print(f"Action space size: {env.observation_space.n}")
# print(f"Action space start value: {env.observation_space.start}")
env.observation_space

<class 'gymnasium.spaces.box.Box'>


Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [16]:
(env.observation_space.shape)

(4,)

In [7]:
observation,_ = env.reset()
observation

array([ 0.03847705, -0.02471477,  0.00235051, -0.02179517], dtype=float32)

In [8]:
next_state, reward, terminated, truncated, info = env.step(1)

In [12]:
next_state

array([ 0.03798275,  0.1703734 ,  0.00191461, -0.31373554], dtype=float32)

In [10]:
from enum import Enum


class Action_Space(int, Enum):
    LEFT = 0
    RIGHT = 1
    NO_ACTION = 2
len(Action_Space)

3

In [11]:
np.random.rand(4)

array([0.72207502, 0.51080984, 0.24993793, 0.93726079])

In [21]:
np.random.rand(*(4,))

array([0.80389116, 0.93223516, 0.14209373, 0.7641221 ])

In [ ]:
TRAINING:bool = False
EVALUATION:bool = True

### Create CartPole environment

In [ ]:
from q_learning_lab.port.environment import create_execute_environment
from q_learning_lab.domain.models.cart_pole_v1_models import get_dnn_structure
from q_learning_lab.domain.deep_q_learn import Reinforcement_DeepLearning
from q_learning_lab.domain.models.cart_pole_v1_models import Params as Cart_Pole_Params
import os

In [ ]:
from q_learning_lab.port.environment import create_execute_environment
from q_learning_lab.domain.models.cart_pole_v1_models import get_dnn_structure
from q_learning_lab.domain.deep_q_learn import Reinforcement_DeepLearning
from q_learning_lab.domain.models.cart_pole_v1_models import Params as Cart_Pole_Params
import os

In [ ]:

from pathlib import Path
params = Cart_Pole_Params(
            total_episodes=100,
            n_max_steps=1000,
            learning_rate=0.7, # learning rate alpha
            gamma=0.618, # discount rate
            epsilon=0.1,
            savefig_folder=Path("_static/img/tutorials/"),
            savemodel_folder=Path("_static/model/test/training"),
            start_epsilon=1.0,  # Starting exploration probability
            min_epsilon=0.05,  # Minimum exploration probability
            decay_rate=0.001,
        )
# Create the environment
env = create_execute_environment(arena="CartPole-v1", params=params)

In [ ]:

if TRAINING:
    dnn_structure = get_dnn_structure(
                input_dim=env.observation_space_dim,
                output_dim=env.action_space_dim,
            )
    deepagent_dict = Reinforcement_DeepLearning.train(
                env=env,
                params=params,
                dnn_structure=dnn_structure,
                is_verbose=True,
                model_name="CartPole-v1-interactive",
            )


In [ ]:
# Load the model from savemodel_folder
from q_learning_lab.port.agent import load_saved_deep_agent

if EVALUATION:
    eval_result = []
    complete_run:int = 0
    my_model_path = os.path.join( str(params.savemodel_folder.joinpath("CartPole-v1_1990")))
    print(my_model_path)
    eval_agent = load_saved_deep_agent(model_path=my_model_path)
    eval_agent.verbose = False
    assert eval_agent.verbose == False
    for episode in range(1, params.total_episodes+1):
        print(f"Episode {episode}")
        reward, complete = eval_agent.play(env=env, max_step=params.n_max_steps)
        eval_result.append(reward)
        if complete:
            complete_run += 1
    print(f"Complete run: {complete_run}")
    stats = np.array(eval_result)
    print(f"Played {episode} {stats.mean():.2f} +/- {stats.std():.2f} (min: {stats.min():.2f}, max: {stats.max():.2f}")
env.close()

In [ ]:
import matplotlib.pyplot as plt
#plot histogram of stats
plt.hist(stats)

In [ ]:
os.getcwd()

In [ ]:

(1,)

In [ ]:
len(Action_Space)